In [5]:
# Dependency
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

In [10]:
# Model info
model_id = "~/wlm2/YanoljaNEXT-Rosetta-20B"
model_id = os.path.expanduser(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.bfloat16,
    device_map="auto",
    max_memory={0: "45GB"},
)
# model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 9/9 [00:09<00:00,  1.02s/it]


In [7]:
target_language = "Korean"
context = {
  "context": "Simple introduction about a tech company.",
  "tone": "Informative and helpful",
  "glossary": {
    "Yanolja NEXT": "야놀자넥스트",
    "travel industry": "여행 산업",
  }
}

system = [f"Translate the user's text to {target_language}."]
for key, value in context.items():
  key_pascal = key.capitalize()
  if isinstance(value, dict):
    system.append(f"{key_pascal}:")
    for f, t in value.items():
      system.append(f"- {f} -> {t}")
  else:
    system.append(f"{key_pascal}: {value}")

system.append("Provide the final translation immediately without any other text.")


In [8]:
source = {
  "company_name": "Yanolja NEXT",
  "description": "Yanolja NEXT is a company that provides cutting-edge "
                 "technology for the global travel industry.",
}

messages = [
    {"role": "system", "content": "\n".join(system)},
    {"role": "user", "content": json.dumps(source, ensure_ascii=False)},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)
# <|start|>instruction<|message|>Translate the user's text to Korean.
# Context: Simple introduction about a tech company.
# Tone: Informative and helpful
# Glossary:
# - Yanolja NEXT -> 야놀자넥스트
# - travel industry -> 여행 산업
# Provide the final translation immediately without any other text.<|end|><|start|>source<|message|>{"company_name": "Yanolja NEXT", "description": "Yanolja NEXT is a company that provides global cutting-edge technology for the travel industry."}<|end|><|start|>translation<|channel|>final<|message|>

<|start|>instruction<|message|>Translate the user's text to Korean.
Context: Simple introduction about a tech company.
Tone: Informative and helpful
Glossary:
- Yanolja NEXT -> 야놀자넥스트
- travel industry -> 여행 산업
Provide the final translation immediately without any other text.<|end|><|start|>source<|message|>{"company_name": "Yanolja NEXT", "description": "Yanolja NEXT is a company that provides cutting-edge technology for the global travel industry."}<|end|><|start|>translation<|channel|>final<|message|>


In [11]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
input_length = inputs["input_ids"].shape[1]

with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
    )

generated_tokens = outputs[0][input_length:]
translation = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(json.dumps(json.loads(translation), indent=2, ensure_ascii=False))

# {
#   "company_name": "야놀자넥스트",
#   "description": "야놀자넥스트는 글로벌 여행 산업에 최첨단 기술을 제공하는 기업입니다."
# }

{
  "company_name": "야놀자넥스트",
  "description": "야놀자넥스트는 글로벌 여행 산업을 위한 최첨단 기술을 제공하는 기업입니다."
}
